In [20]:
import pandas as pd
import numpy as np

from neo4j import GraphDatabase

pd.options.display.max_columns = 50

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
uri = 'bolt://107.23.74.224:7687'
user = 'neo4j'
pwd = 'function-binoculars-twists'

conn = Neo4jConnection(uri=uri, user=user, pwd=pwd)
conn.query("MATCH (n) RETURN COUNT(n)")

[<Record COUNT(n)=281136>]

In [93]:
%%time

query = """MATCH (u:User)-[:POSTED]->(t:Tweet)
           RETURN u.user_key AS user_key,
                  u.id AS u_id,
                  u.name AS name,
                  u.screen_name AS screen_name,
                  u.is_troll AS is_troll,
                  u.created_at AS u_created_at,
                  u.favourites_count AS u_favourites_count,
                  u.followers_count AS followers_count,
                  u.statuses_count AS statuses_count,
                  u.lang AS lang,
                  u.location AS location,
                  u.time_zone AS time_zone,
                  u.verified AS verified,
                  u.description AS description,
                  t.id AS t_id,
                  t.created_at AS t_created_at,
                  t.text AS text,
                  t.favorite_count AS t_favorite_count,
                  t.retweet_count AS retweet_count,
                  t.retweeted AS retweeted
"""

tweets_df = pd.DataFrame([dict(_) for _ in conn.query(query)])
tweets_df.head()

CPU times: user 18.7 s, sys: 596 ms, total: 19.3 s
Wall time: 31 s


,user_key,u_id,name,screen_name,is_troll,u_created_at,u_favourites_count,followers_count,statuses_count,lang,location,time_zone,verified,description,t_id,t_created_at,text,t_favorite_count,retweet_count,retweeted
0,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,764189043152199680,1.471032e+12,Salute brother @iJesseWilliams https://t.co/7...,0.0,0.0,False
1,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,776056376414068738,1.473861e+12,facts https://t.co/OuTm9AdMp3,0.0,0.0,False
2,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,784822121818185728,1.475951e+12,"@eddiebellz not ""mutt"" @realDonaldTrump",0.0,0.0,False
3,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,780602260401299456,NaN,,NaN,NaN,None
4,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,791056262343655425,1.477437e+12,RT @JohnSmithChgo: @AwooMAGA @kushkandy95 @PWM...,0.0,0.0,False


In [94]:
tweets_df.shape

(200833, 20)

In [95]:
%%time

query = """MATCH (t:Tweet)-[:HAS_TAG]->(h:Hashtag)
           RETURN t.id AS t_id,
                  COLLECT(h.tag) AS tags
"""

hashtags_df = pd.DataFrame([dict(_) for _ in conn.query(query)])
hashtags_df.head()

CPU times: user 4.19 s, sys: 212 ms, total: 4.4 s
Wall time: 10 s


,t_id,tags
0,836230963667480576,[thingsdonebymistake]
1,836227574128861184,[thingsdonebymistake]
2,836227673143771136,[thingsdonebymistake]
3,836221435249111041,[thingsdonebymistake]
4,836228095656919040,[thingsdonebymistake]


In [96]:
hashtags_df.shape

(86719, 2)

In [97]:
%%time

query = """MATCH (t:Tweet)-[:MENTIONS]->(u:User)
           RETURN t.id AS t_id,
                  t.text,
                  COLLECT(u.user_key) AS user_mentions
"""

mentions_df = pd.DataFrame([dict(_) for _ in conn.query(query)])
mentions_df.head()

CPU times: user 2.45 s, sys: 96.7 ms, total: 2.55 s
Wall time: 5.43 s


,t_id,t.text,user_mentions
0,795984497967824897,"RT @SCOTTGOHARD: ""you rolling like trump?...ya...","[scottgohard, raekwon]"
1,766661714104422400,RT @JeffreyKahunas: Mysterious murders are not...,[jeffreykahunas]
2,772045505857716224,@JeffreyKahunas Only a blind and dumb fool wou...,[jeffreykahunas]
3,774319033470558208,@WesternWindWes #HillaryClinton is an evil!,[westernwindwes]
4,765196937960235008,RT @IlikeBIGbuttand: Even #ChildrenThinkThat T...,[ilikebigbuttand]


In [98]:
mentions_df.shape

(39514, 3)

In [99]:
%%time

query = """MATCH (t1:Tweet)-[:IN_REPLY_TO]->(t2:Tweet)
           RETURN t2.id AS t_id,
                  COLLECT(t1.id) AS reply_to
"""

replies_df = pd.DataFrame([dict(_) for _ in conn.query(query)])
replies_df.head()

CPU times: user 50.1 ms, sys: 4.13 ms, total: 54.3 ms
Wall time: 415 ms


,t_id,reply_to
0,763800602837123072,[763800657719545856]
1,759506712516784128,"[759506924903751680, 759507042205786112]"
2,773616523990761472,[773616605259595776]
3,759164202611306496,[759165124716429312]
4,793874305511780352,[793874529017880577]


In [100]:
replies_df.shape

(516, 2)

In [101]:
%%time

query = """MATCH (t:Tweet)-[:HAS_LINK]->(u:URL)
           WHERE u.expanded_url <> ""
           RETURN t.id AS t_id,
                  COLLECT(u.expanded_url) AS urls
"""

url_df = pd.DataFrame([dict(_) for _ in conn.query(query)])
url_df.head()

CPU times: user 1.27 s, sys: 55.8 ms, total: 1.33 s
Wall time: 3.12 s


,t_id,urls
0,777594647875059712,[http://ln.is/twitchy.com/loriz-31/3yafU]
1,782408661389840384,[https://twitter.com/i/web/status/782408661389...
2,777859822679158784,[http://freedomoutpost.com/2013/02/the-use-of-...
3,781999544502521857,[http://www.informationliberation.com/?id=55568]
4,777238119166775297,[https://twitter.com/Harlan/status/77714580969...


In [102]:
url_df.shape

(25381, 2)

In [119]:
%%time

query = """MATCH (t1:Tweet)-[:RETWEETED]->(t2:Tweet)
           RETURN t1.id AS t_id,
                  COLLECT(t2.id) AS retweet
"""

retweets_df = pd.DataFrame([dict(_) for _ in conn.query(query)])
retweets_df.head()

CPU times: user 2.4 s, sys: 120 ms, total: 2.52 s
Wall time: 5.49 s


,t_id,retweet
0,760786317374545920,[760628814871310337]
1,784313832487256064,[784310979710492672]
2,777594647875059712,[777591478206029824]
3,785586729579196416,[785326419509411840]
4,795661161282891776,[795463782671798272]


In [120]:
how = 'left'

rtt_df = pd.merge(left=tweets_df, right=mentions_df, how=how, on='t_id', suffixes=('_left','_right'))
rtt_df = pd.merge(left=rtt_df, right=hashtags_df, how=how, on='t_id', suffixes=('_left','_right'))
rtt_df = pd.merge(left=rtt_df, right=replies_df, how=how, on='t_id', suffixes=('_left','_right'))
rtt_df = pd.merge(left=rtt_df, right=url_df, how=how, on='t_id', suffixes=('_left','_right'))
rtt_df = pd.merge(left=rtt_df, right=retweets_df, how=how, on='t_id', suffixes=('_left','_right'))
rtt_df.head()

,user_key,u_id,name,screen_name,is_troll,u_created_at,u_favourites_count,followers_count,statuses_count,lang,location,time_zone,verified,description,t_id,t_created_at,text,t_favorite_count,retweet_count,retweeted,t.text,user_mentions,tags,reply_to,urls,retweet
0,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,764189043152199680,1.471032e+12,Salute brother @iJesseWilliams https://t.co/7...,0.0,0.0,False,Salute brother @iJesseWilliams https://t.co/7...,[ijessewilliams],NaN,NaN,[https://twitter.com/ijessewilliams/status/764...,NaN
1,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,776056376414068738,1.473861e+12,facts https://t.co/OuTm9AdMp3,0.0,0.0,False,NaN,NaN,NaN,NaN,[https://twitter.com/abc7/status/7759306333861...,NaN
2,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,784822121818185728,1.475951e+12,"@eddiebellz not ""mutt"" @realDonaldTrump",0.0,0.0,False,"@eddiebellz not ""mutt"" @realDonaldTrump",[realdonaldtrump],NaN,NaN,NaN,NaN
3,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,780602260401299456,NaN,,NaN,NaN,None,NaN,NaN,NaN,[780607504128638976],NaN,NaN
4,scottgohard,100345056,#Ezekiel2517✨...,SCOTTGOHARD,1,Tue Dec 29 23:15:22 +0000 2009,2774.0,1053.0,31858.0,en,still ⬆️Block⤵️Corner⬇️street,,False,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,791056262343655425,1.477437e+12,RT @JohnSmithChgo: @AwooMAGA @kushkandy95 @PWM...,0.0,0.0,False,RT @JohnSmithChgo: @AwooMAGA @kushkandy95 @PWM...,"[jsavite, johnsmithchgo, kushkandy95, awoomaga...",NaN,NaN,NaN,[791055623240708096]


In [116]:
rtt_df.shape

(200833, 25)

In [121]:
rtt_df.to_csv('./rtt.csv', header=True, index=False)